</center><img src="coverage_workflow_image.png"  width="1000" height="300"/></center>

In [1]:
# import required python libraries
#import geopandas as gpd  # for geo-data structure
import numpy as np  # for some operation better done with numpy
import matplotlib.pyplot as plt  # for creating plots
#import shapely.geometry  # for geometry data crunching
#from shapely.geometry import Point  # specifically for point geometry data manipulation
#import keplergl
#from keplergl import KeplerGl
import datacompy
import pandas as pd
crs = {'init': 'epsg:4326'}
#from shapely.geometry import Point
import mpl_toolkits


In [11]:
# bring in the 3 to 4 different coverage dataset from FB, GSMA, ITU and Ericsson as well as the master school_geolocation

In [58]:
d_f = pd.read_csv('./data/SierraLeone_Connectivity.csv',low_memory=False,encoding='utf-8')
d_i = pd.read_csv('./data/sierra_leone_school_coverage_ITU.csv',low_memory=False,encoding='utf-8')
#d_e = pd.read_csv('./data/Rwanda_Ericsson_07_06_2021.csv',low_memory=False,encoding='utf-8')
d_g = pd.read_csv('./data/coverage_Sierra_Leone_GSMA_V2.csv',low_memory=False,encoding='utf-8')
s_master = pd.read_csv('./data/SL_geolocation_master.csv',low_memory=False,encoding='utf-8')

In [59]:
d_f.head(2)

,tile_bigint,clean_lat,clean_lon,uuid,country,source,giga_school_id,source_school_id,name,education_level,...,school_archive,electricity_availability,connectivity_availability,num_students,num_teachers,num_computers,computer_lab,percent_2g,percent_3g,percent_4g
0,2.090000e+15,8.972037,-12.91269,9ff31a81-67df-3629-bbcc-2dbc72e93c32,Sierra Leone,government,G_SL_000094,NaN,KDC PRESCHOOL TAINTAFOR,Primary,...,No,No,No,70,1,NaN,No,11.02,74.02,14.96
1,2.120000e+15,8.578695,-10.71954,68f65b94-2664-36b4-ab4c-ce3ce522fd9b,Sierra Leone,government,G_SL_000080,NaN,K.D.E.C PRIMARY SCHOOL WAYIAH,Primary,...,No,No,No,252,3,NaN,No,10.94,74.22,14.84


In [14]:
# some colunm names standardization

In [15]:
#d_e.rename(columns={'SCHOOL_CODE':'school_id', 'SCHOOL_NAME':'school_name'}, inplace=True)
#d_i.rename(columns={'giga_schoo': 'giga_school_id','source_sch':'school_id', 'name':'school_name'}, inplace=True)
#d_g.rename(columns={'giga_schoo': 'giga_school_id'}, inplace=True)

In [16]:
# cross-profiles the sources with master data for completeness and correctness

In [60]:
# create a separate dfs
master = s_master.copy()
fb = d_f.copy()
#ec = d_e.copy()
itu = d_i.copy()
gsma = d_g.copy()
# for facebook
cross_profile_fb_master = datacompy.Compare(
master,
fb,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Facebook'
)
#for ericsson
'''''
cross_profile_e_master = datacompy.Compare(
master,
ec,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='Ericsson'
)
'''''
# for ITU
cross_profile_itu_master = datacompy.Compare(
master,
itu,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='ITU'
)
# for GSMA
cross_profile_gsma_master = datacompy.Compare(
master,
gsma,
join_columns='giga_school_id',  
df1_name='Master', 
df2_name='GSMA'
)
print(cross_profile_fb_master.report())
#print(cross_profile_e_master.report())
print(cross_profile_itu_master.report())
print(cross_profile_gsma_master.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0    Master       20  10893
1  Facebook       26   7849

Column Summary
--------------

Number of columns in common: 19
Number of columns in Master but not in Facebook: 1
Number of columns in Facebook but not in Master: 7

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 7,849
Number of rows in Master but not in Facebook: 3,044
Number of rows in Facebook but not in Master: 0

Number of rows with some compared columns unequal: 7,849
Number of rows with all compared columns equal: 0

Column Comparison
-----------------

Number of columns compared with some values unequal: 2
Number of columns compared with all values equal: 17
Total number of values which compare unequal: 15,179

Columns with Unequal Values or Types
------------------------------------

             Column Mas

In [18]:
# they usually don't come with "coverage" colunms, we create it at source level using the following codes

In [61]:
#d_e['coverage'] = np.where((d_e['2G_coverage']== 'None') & (d_e['3G_coverage']== 'None')& (d_e['4G_coverage']== 'None'), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g'].isnull()) & (d_f['percent_3g'].isnull())& (d_f['percent_4g'].isnull()), 'NO', 'YES')
d_f['coverage'] = np.where((d_f['percent_2g']<=0) & (d_f['percent_3g']<=0) & (d_f['percent_4g']<=0), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G'].isnull()) & (d_i['3G'].isnull())& (d_i['4G'].isnull()), 'NO', 'YES')
d_i['coverage'] = np.where((d_i['2G']<=0) & (d_i['3G']<=0) & (d_i['4G']<=0), 'NO', 'YES')
d_g['coverage'] = np.where((d_g['2G'].isnull()) & (d_g['3G'].isnull())& (d_g['4G'].isnull()), 'NO', 'YES')
d_g['coverage'] = np.where((d_g['2G']<=0) & (d_g['3G']<=0) & (d_g['4G']<=0), 'NO', 'YES')

In [62]:
# create the 2G, 3G and 4G coverage columns with binary values at source level using the following codes

In [63]:
d_g['2G_coverage'] = np.where((d_g['2G'] > 0), 'YES', 'NO')
d_g['3G_coverage'] = np.where((d_g['3G'] > 0), 'YES', 'NO')
d_g['4G_coverage'] = np.where((d_g['4G'] > 0), 'YES', 'NO')

In [64]:
d_f['2G_coverage'] = np.where((d_f['percent_2g'] > 0), 'YES', 'NO')
d_f['3G_coverage'] = np.where((d_f['percent_3g'] > 0), 'YES', 'NO')
d_f['4G_coverage'] = np.where((d_f['percent_4g'] > 0), 'YES', 'NO')

In [65]:
d_i['2G_coverage'] = np.where((d_i['2G'] >=1), 'YES', 'NO')
d_i['3G_coverage'] = np.where((d_i['3G'] == 1), 'YES', 'NO')
d_i['4G_coverage'] = np.where((d_i['4G'] ==1), 'YES', 'NO')

In [23]:
#d_e.rename(columns={'2G_coverage':'2G_coverage_quality', '3G_coverage':'3G_coverage_quality', '4G_coverage': '4G_coverage_quality'}, inplace=True)

In [24]:
'''''
d_e['2G_coverage'] = np.where((d_e['2G_coverage_quality'] != 'None'), 'YES', 'NO')
d_e['3G_coverage'] = np.where((d_e['3G_coverage_quality'] != 'None'), 'YES', 'NO')
d_e['4G_coverage'] = np.where((d_e['4G_coverage_quality'] != 'None'), 'YES', 'NO')
d_i['2G_coverage'] = np.where((d_i['2G'] >=1), 'YES', 'NO')
d_i['3G_coverage'] = np.where((d_i['3G'] == 1), 'YES', 'NO')
d_i['4G_coverage'] = np.where((d_i['4G'] ==1), 'YES', 'NO')

In [25]:
# they usually don't come with "coverage_type" colunms, we create it at source level using the following function and codes

In [66]:
def coverage_type(df):
    if df['4G_coverage'] == 'YES':
        return "4G"
    elif df['3G_coverage'] == 'YES':
        return "3G"
    elif df['2G_coverage'] == 'YES':
        return "2G"
    else:
        return "None"

In [67]:
d_f['coverage_type'] = d_f.apply((lambda x: coverage_type(x)),axis=1)
#d_e['coverage_type'] = d_e.apply((lambda x: coverage_type(x)),axis=1)
d_i['coverage_type'] = d_i.apply((lambda x: coverage_type(x)),axis=1)
d_g['coverage_type'] = d_g.apply((lambda x: coverage_type(x)),axis=1)

In [41]:
# isolating the needed colunms from the rest

In [71]:
d_fc.head()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,G_SL_000094,YES,4G,YES,YES,YES
1,G_SL_000080,YES,4G,YES,YES,YES
2,G_SL_000033,YES,4G,YES,YES,YES
3,G_SL_000104,YES,4G,YES,YES,YES
4,G_SL_000240,YES,4G,YES,YES,YES


In [68]:
features = ['giga_school_id','coverage','coverage_type','2G_coverage','3G_coverage','4G_coverage'] 

In [69]:
d_fc = d_f[features]
#d_ec = d_e[features]
d_it = d_i[features]
d_gs = d_g[features]

In [31]:
# more sanity checks for nan on giga_school_id

In [70]:
d_fc = d_fc[d_fc['giga_school_id'].notna()]
#d_ec = d_ec[d_ec['giga_school_id'].notna()]
d_it = d_it[d_it['giga_school_id'].notna()]
d_gs = d_gs[d_gs['giga_school_id'].notna()]

In [33]:
# visually comparing the sources for matches and mismatches across the columns

In [72]:
df_all = pd.concat([d_fc.set_index('giga_school_id'), d_gs.set_index('giga_school_id'),d_it.set_index('giga_school_id')], 
                   axis='columns', keys=['Facebook', 'GSMA', 'ITU'])
df_all.head()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Facebook                                                    \
            coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
G_SL_000001      YES            4G         YES         YES         YES   
G_SL_000002      YES            4G         YES         YES         YES   
G_SL_000003      YES            4G         YES         YES         YES   
G_SL_000004      NaN           NaN         NaN         NaN         NaN   
G_SL_000005      NaN           NaN         NaN         NaN         NaN   

                GSMA                                                    \
            coverage coverage_type 2G_coverage 3G_coverage 4G_coverage   
G_SL_000001      YES            2G         YES          NO          NO   
G_SL_000002      YES          None          NO          NO          NO   
G_SL_000003      YES            2G         YES          NO          NO   
G_SL_000004      YES            2G         YES          NO          NO   
G_SL_000005      YES          None          NO          NO          NO   

                 ITU                                                    
            coverage coverage_type 2G_coverage 3G_coverage 4G_coverage  
G_SL_000001      YES            2G         YES          NO          NO  
G_SL_000002      YES            2G         YES          NO          NO  
G_SL_000003      YES            2G         YES          NO          NO  
G_SL_000004      YES            2G         YES          NO          NO  
G_SL_000005       NO          None          NO          NO          NO

In [73]:
df_final = df_all.swaplevel(axis='columns')[d_fc.columns[1:]]
df_final.head()

coverage           coverage_type             2G_coverage       \
            Facebook GSMA  ITU      Facebook  GSMA   ITU    Facebook GSMA   
G_SL_000001      YES  YES  YES            4G    2G    2G         YES  YES   
G_SL_000002      YES  YES  YES            4G  None    2G         YES   NO   
G_SL_000003      YES  YES  YES            4G    2G    2G         YES  YES   
G_SL_000004      NaN  YES  YES           NaN    2G    2G         NaN  YES   
G_SL_000005      NaN  YES   NO           NaN  None  None         NaN   NO   

                 3G_coverage          4G_coverage           
             ITU    Facebook GSMA ITU    Facebook GSMA ITU  
G_SL_000001  YES         YES   NO  NO         YES   NO  NO  
G_SL_000002  YES         YES   NO  NO         YES   NO  NO  
G_SL_000003  YES         YES   NO  NO         YES   NO  NO  
G_SL_000004  YES         NaN   NO  NO         NaN   NO  NO  
G_SL_000005   NO         NaN   NO  NO         NaN   NO  NO

In [74]:
def highlight_diff(data, color='lightcoral'):
    attr = 'background-color: {}'.format(color)
    other = data.xs('Facebook', axis='columns', level=-1)
    return pd.DataFrame(np.where(data.ne(other, level=0), attr, ''),
                        index=data.index, columns=data.columns)

#df_final.style.apply(highlight_diff, axis=None)

In [37]:
# coverage comparison report

In [79]:
# create a separate dfs
fbc = d_fc.copy()
#ecc = d_ec.copy()
ituc = d_it.copy()
gsm = d_gs.copy()

# Facebook&ITU
cross_profile_fb_itu = datacompy.Compare(
fbc,
ituc,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='ITU'
)

cross_profile_facebook_gsma = datacompy.Compare(
fbc,
gsm,
join_columns='giga_school_id',  
df1_name='Facebook', 
df2_name='GSMA'
)


#print(cross_profile_fb_ec.report())
print(cross_profile_fb_itu.report())
#print(cross_profile_Ericsson_itu.report())
print(cross_profile_facebook_gsma.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns   Rows
0  Facebook        6   7849
1       ITU        6  10893

Column Summary
--------------

Number of columns in common: 6
Number of columns in Facebook but not in ITU: 0
Number of columns in ITU but not in Facebook: 0

Row Summary
-----------

Matched on: giga_school_id
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 7,849
Number of rows in Facebook but not in ITU: 0
Number of rows in ITU but not in Facebook: 3,044

Number of rows with some compared columns unequal: 4,362
Number of rows with all compared columns equal: 3,487

Column Comparison
-----------------

Number of columns compared with some values unequal: 5
Number of columns compared with all values equal: 1
Total number of values which compare unequal: 13,159

Columns with Unequal Values or Types
------------------------------------

          Column Facebook dtype I

In [39]:
# merging and selecting the most frequent values across the sources on each row and colunm
# creating a master coverage dataset and merging with the master school location dataset

In [104]:
df_c = pd.concat([d_fc,d_gs,d_it])
coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x:x.value_counts().index[0])


In [105]:
coverage.tail()

,giga_school_id,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
10888,G_SL_010889,YES,4G,YES,YES,YES
10889,G_SL_010890,YES,4G,YES,YES,YES
10890,G_SL_010891,YES,4G,YES,YES,YES
10891,G_SL_010892,YES,4G,YES,YES,YES
10892,G_SL_010893,YES,4G,YES,YES,YES


In [97]:
#coverage.loc[coverage['coverage_type'] == 'None', 'coverage_type'] = d_gs.coverage_type

In [ ]:
''''
#Main
import statistics as stats
df_c = pd.concat([d_fc,d_gs,d_it])
coverage = df_c.groupby(['giga_school_id'],as_index=False).agg(lambda x: stats.mode(x)[0][0])
coverage.loc[coverage['coverage'] == 'Y', 'coverage'] = 'YES'
coverage.loc[coverage['coverage'] == 'N', 'coverage'] = 'NO'
coverage.loc[coverage['2G_coverage'] == 'Y', '2G_coverage'] = 'YES'
coverage.loc[coverage['2G_coverage'] == 'N', '2G_coverage'] = 'NO'
coverage.loc[coverage['3G_coverage'] == 'Y', '3G_coverage'] = 'YES'
coverage.loc[coverage['3G_coverage'] == 'N', '3G_coverage'] = 'NO'
coverage.loc[coverage['4G_coverage'] == 'Y', '4G_coverage'] = 'YES'
coverage.loc[coverage['4G_coverage'] == 'N', '4G_coverage'] = 'NO'

coverage.loc[coverage['coverage_type'] == '4', 'coverage_type'] = '4G'
coverage.loc[coverage['coverage_type'] == '3', 'coverage_type'] = '3G'
coverage.loc[coverage['coverage_type'] == '2', 'coverage_type'] = '2G'
coverage.loc[coverage['coverage_type'] == 'N', 'coverage_type'] = 'None'

coverage.loc[coverage['coverage_type'] == 'None', 'coverage_type'] = d_gs.coverage_type

In [99]:
master.head()

,uuid,country,source,giga_school_id,school_id,name,education_level,lat,lon,hex8,dq_score,source_timestamp,master_timestamp,school_archive,electricity_availability,connectivity_availability,num_students,num_teachers,num_computers,computer_lab
0,b4218644-52aa-3da9-8bfd-b60ef9e60912,Sierra Leone,government,G_SL_000001,NaN,BADRUDEEN ISLAMIC PRIMARY SCHOOL,Primary,7.988612,-10.64796,8875b084a9fffff,0.67,2020,2021-08-16 23:04:02.701831,No,No,No,117.0,4.0,NaN,No
1,84b52390-a1a8-3e1d-b2f3-1bebde50f350,Sierra Leone,government,G_SL_000002,NaN,KAILAHUNTER DISTRICT EDUCATION COMMITTEE PRIMA...,Primary,7.822542,-10.92325,8875b00213fffff,0.67,2020,2021-08-16 23:04:02.701831,No,No,No,111.0,3.0,NaN,No
2,70074963-6cd0-3cd8-a574-78e5acbfefb8,Sierra Leone,government,G_SL_000003,NaN,BADRUDIEN ISLAMIC PRIMARY SCHOOL,Primary,8.282627,-10.37537,8875b5533dfffff,0.67,2020,2021-08-16 23:04:02.701831,No,No,No,190.0,4.0,NaN,No
3,e734f64f-4ad4-3a17-9d55-9e032a6cca5e,Sierra Leone,government,G_SL_000004,NaN,KAILAHUN DISTRICT EDUCATION COMMITTEE PRE-...,Primary,8.283335,-10.37064,8875b55339fffff,0.67,2020,2021-08-16 23:04:02.701831,No,No,No,55.0,2.0,NaN,No
4,bbc80ca5-4e5f-3896-b1f3-6c2f87f5b61e,Sierra Leone,government,G_SL_000005,NaN,KAILAHUN DISTRICT EDUCATION COMMITTEE PRIM...,Primary,8.206346,-10.34603,8875b550a5fffff,0.67,2020,2021-08-16 23:04:02.701831,No,No,No,133.0,3.0,NaN,No


In [115]:
master_w_coverage = master.merge(coverage, how="left", on=["giga_school_id"])
master_w_coverage.to_csv('./data/SL_master_w_coverage.csv', encoding='utf-8')
master_w_coverage_connectivity = master_w_coverage.copy()
#master_w_coverage_connectivity['connectivity_availability'] = np.where((master_w_coverage_connectivity['connectivity_type'] != 'None'), 'YES', 'NO')
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'connectivity_availability	'] = 'NO'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage'] == 'NO', 'coverage_type'] = 'None'
master_w_coverage_connectivity.loc[master_w_coverage_connectivity['coverage_type'] == 'None', 'coverage'] = 'NO'
master_w_coverage_connectivity.to_csv('./data/SL_master_w_coverage_connectivity.csv', encoding='utf-8')

In [113]:
master_w_coverage.head(2)

,uuid,country,source,giga_school_id,school_id,name,education_level,lat,lon,hex8,...,connectivity_availability,num_students,num_teachers,num_computers,computer_lab,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage
0,b4218644-52aa-3da9-8bfd-b60ef9e60912,Sierra Leone,government,G_SL_000001,NaN,BADRUDEEN ISLAMIC PRIMARY SCHOOL,Primary,7.988612,-10.64796,8875b084a9fffff,...,No,117.0,4.0,NaN,No,YES,2G,YES,NO,NO
1,84b52390-a1a8-3e1d-b2f3-1bebde50f350,Sierra Leone,government,G_SL_000002,NaN,KAILAHUNTER DISTRICT EDUCATION COMMITTEE PRIMA...,Primary,7.822542,-10.92325,8875b00213fffff,...,No,111.0,3.0,NaN,No,YES,4G,YES,NO,NO


In [112]:
master_w_coverage_connectivity[master_w_coverage_connectivity['coverage_type']=='None'].head(10)

,uuid,country,source,giga_school_id,school_id,name,education_level,lat,lon,hex8,...,num_students,num_teachers,num_computers,computer_lab,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage,connectivity_availability\t
4,bbc80ca5-4e5f-3896-b1f3-6c2f87f5b61e,Sierra Leone,government,G_SL_000005,NaN,KAILAHUN DISTRICT EDUCATION COMMITTEE PRIM...,Primary,8.206346,-10.34603,8875b550a5fffff,...,133.0,3.0,NaN,No,NO,None,NO,NO,NO,NaN
6,f1e92c26-d024-3374-bc2c-815e815a6105,Sierra Leone,government,G_SL_000007,NaN,MAKPADU COMMUNITY PRIMARY SCHOOL,Primary,8.235952,-10.32845,8875b55013fffff,...,200.0,5.0,NaN,No,NO,None,NO,NO,NO,NaN
18,e4082e4d-978f-3450-bd6f-048ab6656dd5,Sierra Leone,government,G_SL_000019,NaN,UNITED ISLAMIC PRIMARY SCHOOL,Primary,7.660082,-11.18757,8875b07329fffff,...,58.0,2.0,NaN,No,NO,None,NO,NO,NO,NaN
46,40133a2e-2304-3cc3-82ab-9fa341eba58a,Sierra Leone,government,G_SL_000047,NaN,CHARITABLE COMMUNITY PRIMARY SCHOOL,Primary,7.620267,-11.32312,8875b07b61fffff,...,72.0,3.0,NaN,No,NO,None,NO,NO,NO,NaN
56,900af572-ffde-3f3a-b787-ff4d1d48b749,Sierra Leone,government,G_SL_000057,NaN,ROMAN CATHOLIC PRIMARY SCHOOL KONGOHUN,Primary,7.545238,-11.08949,8875b07509fffff,...,81.0,1.0,NaN,No,NO,None,NO,NO,NO,NaN
74,74d0e679-5511-355c-b533-d32999450b0a,Sierra Leone,government,G_SL_000075,NaN,KONO DISTRICT EDUCATION COMMITTEE PRIMARY SCHOOL,Primary,8.828598,-10.84494,8875b41b03fffff,...,125.0,2.0,NaN,No,NO,None,NO,NO,NO,NaN
77,b1e4d3da-099e-373e-a1f5-3665e3040047,Sierra Leone,government,G_SL_000078,NaN,GOSPEL HARMONY AGRICULTURAL VOCATIONAL JUNIOR ...,Secondary,8.617138,-10.72268,8875b40e03fffff,...,61.0,4.0,NaN,No,NO,None,NO,NO,NO,NaN
78,21f491a4-897f-3f4a-88ea-589f93f7e03a,Sierra Leone,government,G_SL_000079,NaN,K.D.E.C PRIMARY SCHOOL FAAMA,Primary,8.654231,-10.73059,8875b4019bfffff,...,85.0,3.0,NaN,No,NO,None,NO,NO,NO,NaN
79,68f65b94-2664-36b4-ab4c-ce3ce522fd9b,Sierra Leone,government,G_SL_000080,NaN,K.D.E.C PRIMARY SCHOOL WAYIAH,Primary,8.578695,-10.71954,8875b40ea9fffff,...,252.0,3.0,NaN,No,NO,None,NO,NO,NO,NaN
111,f8e59063-d779-39ac-a160-aed490e7b7f5,Sierra Leone,government,G_SL_000112,NaN,IAM PRIMARY SCHOOL MASOWA,Primary,9.416848,-12.12524,8854e82d31fffff,...,106.0,2.0,NaN,No,NO,None,NO,NO,NO,NaN


In [116]:
master_w_coverage_connectivity.head()

,uuid,country,source,giga_school_id,school_id,name,education_level,lat,lon,hex8,...,num_students,num_teachers,num_computers,computer_lab,coverage,coverage_type,2G_coverage,3G_coverage,4G_coverage,connectivity_availability\t
0,b4218644-52aa-3da9-8bfd-b60ef9e60912,Sierra Leone,government,G_SL_000001,NaN,BADRUDEEN ISLAMIC PRIMARY SCHOOL,Primary,7.988612,-10.64796,8875b084a9fffff,...,117.0,4.0,NaN,No,YES,2G,YES,NO,NO,NaN
1,84b52390-a1a8-3e1d-b2f3-1bebde50f350,Sierra Leone,government,G_SL_000002,NaN,KAILAHUNTER DISTRICT EDUCATION COMMITTEE PRIMA...,Primary,7.822542,-10.92325,8875b00213fffff,...,111.0,3.0,NaN,No,YES,4G,YES,NO,NO,NaN
2,70074963-6cd0-3cd8-a574-78e5acbfefb8,Sierra Leone,government,G_SL_000003,NaN,BADRUDIEN ISLAMIC PRIMARY SCHOOL,Primary,8.282627,-10.37537,8875b5533dfffff,...,190.0,4.0,NaN,No,YES,2G,YES,NO,NO,NaN
3,e734f64f-4ad4-3a17-9d55-9e032a6cca5e,Sierra Leone,government,G_SL_000004,NaN,KAILAHUN DISTRICT EDUCATION COMMITTEE PRE-...,Primary,8.283335,-10.37064,8875b55339fffff,...,55.0,2.0,NaN,No,YES,2G,YES,NO,NO,NaN
4,bbc80ca5-4e5f-3896-b1f3-6c2f87f5b61e,Sierra Leone,government,G_SL_000005,NaN,KAILAHUN DISTRICT EDUCATION COMMITTEE PRIM...,Primary,8.206346,-10.34603,8875b550a5fffff,...,133.0,3.0,NaN,No,NO,None,NO,NO,NO,NaN


In [114]:
master_w_coverage_connectivity['coverage'].value_counts()

YES    10246
NO       647
Name: coverage, dtype: int64